In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from pymongo.mongo_client import MongoClient
from pymongo.database import Database

In [2]:
def signleTon(class_instance):
    instances = {}
    
    def get_instances(*args, **kwargs):
        if class_instance not in instances:
            instances[class_instance] = class_instance(*args, **kwargs)
        return instances[class_instance]
    
    return get_instances

@signleTon
class MongoDBConnection:
    
    def __init__(self, url_database, db_connection):
        self._client = MongoClient(url_database)
        self._db = self._client[db_connection]
    
    def get_database(self) -> Database:
        return self._db

In [3]:
# call data

MONGODB_URL = "mongodb://localhost:27017/"
DATABASE_NAME = "DATABASE_SHOPEE"

def get_data():
    db = MongoDBConnection(MONGODB_URL, DATABASE_NAME).get_database()
    collection = db['products']
    data = pd.DataFrame(list(collection.find()))
    return data

In [4]:
df = get_data()

In [5]:
df.head()

,_id,id,name,type,current_seller,images,short_url,thumbnail_url,short_description,description,rating_average,quantity_sold,embedding
0,67025a168a85cf6b261b7980,263166362,Bài Tập Tiếng Anh 8 ( Theo chương trình GDPT M...,simple,94000,[{'base_url': 'https://salt.tikicdn.com/ts/pro...,https://tiki.vn/product-p263166362.html?spid=2...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Các em học sinh thân mến!Chúng tôi biên soạn B...,Các em học sinh thân mến!\nChúng tôi biên soạn...,5.0,119,"[[4.627774160326226e-06, -0.000711660424713045..."
1,67025a1a8a85cf6b261b7981,263693271,Bảng trắng có chân gấp Flipchart F3 PolyTaiwan...,simple,1100000,[{'base_url': 'https://salt.tikicdn.com/ts/pro...,https://tiki.vn/product-p263693271.html?spid=2...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Bảng Trắng Có Chân Gấp Flipchart của thương hi...,Bảng Trắng Có Chân Gấp Flipchart của thương hi...,0.0,6,"[[-0.004785317927598953, -0.013490630313754082..."
2,67025a1d8a85cf6b261b7982,262977989,Kế Toán Vỉa Hè - Thực Hành Báo Cáo Tài Chính C...,simple,169000,[{'base_url': 'https://salt.tikicdn.com/ts/pro...,https://tiki.vn/product-p262977989.html?spid=2...,https://salt.tikicdn.com/cache/280x280/ts/prod...,BIẾN KẾ TOÁN KHÔ KHAN TRỞ THÀNH TRÒ CHƠI CON T...,BIẾN KẾ TOÁN KHÔ KHAN TRỞ THÀNH TRÒ CHƠI CON T...,5.0,13264,"[[-0.02545916847884655, -0.0020544526632875204..."
3,67025a228a85cf6b261b7983,275025208,Sách Combo 2 Cuốn : Tư Duy Ngược + Tư Duy Mở (...,simple,138000,[{'base_url': 'https://salt.tikicdn.com/ts/pro...,https://tiki.vn/product-p275025208.html?spid=2...,https://salt.tikicdn.com/cache/280x280/ts/prod...,THÔNG TIN TÁC PHẨMTên tác phẩm: Combo 2 Cuốn T...,THÔNG TIN TÁC PHẨM\n\nTên tác phẩm: Combo 2 Cu...,4.8,2657,"[[-0.0005972246290184557, 0.00408864114433527,..."
4,67025a268a85cf6b261b7984,270546273,Combo Tư Duy Ngược + Tư Duy Mở (Nguyễn Anh Dũn...,simple,135000,[{'base_url': 'https://salt.tikicdn.com/ts/pro...,https://tiki.vn/product-p270546273.html?spid=2...,https://salt.tikicdn.com/cache/280x280/ts/prod...,THÔNG TIN TÁC PHẨMTên tác phẩm: Combo 2 Cuốn T...,THÔNG TIN TÁC PHẨM\n\nTên tác phẩm: Combo 2 Cu...,4.7,1767,"[[-0.009435735642910004, -0.007636362221091986..."


In [6]:
# load stopword
def load_stopwords(file_path: str):
    """
    Đọc stopwords từ file và trả về dưới dạng một danh sách.
    """
    stopwords = None
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = [line.strip() for line in file]
    return stopwords

stopWord = load_stopwords("D:\WorkSpace\RAG\src\services\stopWord.txt")

In [7]:
# clearn text and characters specified

def remove_numbers_and_special_chars(text):
    """
    Hàm loại bỏ số và các ký tự đặc biệt, giữ lại chữ cái có dấu trong văn bản tiếng Việt.
    """
    # Sử dụng regex để loại bỏ số và ký tự đặc biệt nhưng giữ lại dấu tiếng Việt
    text = re.sub(r'[0-9]', '', text)  # Loại bỏ số
    text = re.sub(r'[^\w\sÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơ'
                  r'ẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪ'
                  r'ỬỮỰỲỴÝỶỸỳỵýỷỹ]', '', text)  # Loại bỏ ký tự đặc biệt nhưng giữ lại dấu tiếng Việt
    text = re.sub(r'\s+', ' ', text)  # Loại bỏ khoảng trắng thừa
    text = text.lower()
    return text.strip()


In [8]:
text = df['description'][0]
print(text)

Các em học sinh thân mến!
Chúng tôi biên soạn Bài tập Tiếng Anh 8 – Không đáp án trên cơ sở bám sát Chương trình môn Tiếng Anh được ban hành kèm theo Thông tư số 32/2008/TT-BGDĐT ngày 26/12/2018, và sách giáo khoa Tiếng Anh 8 mới, được giảng dạy đại trà từ năm học 2023-2024.
Bài tập Tiếng Anh 8 – Không đáp án gồm một bài ôn tập củng cố kiến thức đã học (Starter Unit), và 8 đơn vị bài học chính: Fads and fashions, Sensations, Adventure, Material world, Years ahead, Learn, Big ideas, On screen và sau mỗi đơn vị bài học đều có bài ôn tập Review.
Bài tập dành cho mỗi đơn vị bài học chính gồm 6 phần: Vocabulary, Language focus, Reading, Pronunciation, Speaking, và Writing.
Các bài Review (Vocabulary, Language focus, Reading, Speaking và Writing) giúp học sinh tự kiểm tra những nội dung kiến thức và rèn luyện các kĩ năng sau mỗi đơn vị bài học.
Các bài tập trong Bài tập Tiếng Anh 8 – Không đáp án đa dạng, phong phú, bám sát từ vựng, ngữ pháp và chủ đề trong sách giáo khoa tiếng Anh theo chươ

In [9]:
text_remove_special_characters = remove_numbers_and_special_chars(text)
print(text_remove_special_characters)

các em học sinh thân mến chúng tôi biên soạn bài tập tiếng anh không đáp án trên cơ sở bám sát chương trình môn tiếng anh được ban hành kèm theo thông tư số ttbgdđt ngày và sách giáo khoa tiếng anh mới được giảng dạy đại trà từ năm học bài tập tiếng anh không đáp án gồm một bài ôn tập củng cố kiến thức đã học starter unit và đơn vị bài học chính fads and fashions sensations adventure material world years ahead learn big ideas on screen và sau mỗi đơn vị bài học đều có bài ôn tập review bài tập dành cho mỗi đơn vị bài học chính gồm phần vocabulary language focus reading pronunciation speaking và writing các bài review vocabulary language focus reading speaking và writing giúp học sinh tự kiểm tra những nội dung kiến thức và rèn luyện các kĩ năng sau mỗi đơn vị bài học các bài tập trong bài tập tiếng anh không đáp án đa dạng phong phú bám sát từ vựng ngữ pháp và chủ đề trong sách giáo khoa tiếng anh theo chương trình mới chúng tôi hi vọng rằng bài tập tiếng anh không đáp án sẽ là một tài

In [10]:
def clean_text(text: str, stopwords: list):
    """
    Xử lý văn bản tiếng Việt: Loại bỏ các ký tự không mong muốn và stopwords dạng từ đơn hoặc cụm từ.
    """
    # laoij bo cac ký tự đặc biệt
    text = remove_numbers_and_special_chars(text)
    # Loại bỏ các ký tự đặc biệt nhưng giữ lại dấu tiếng Việt
    text = re.sub(r'[^\w\s]', '', text)

    # Đưa về chữ thường
    text = text.lower()

    # Loại bỏ stopwords
    for stopword in stopwords:
        # Sử dụng re.sub để loại bỏ stopword (dạng từ hoặc cụm từ) khỏi văn bản
        pattern = r'\b' + re.escape(stopword) + r'\b'
        text = re.sub(pattern, '', text)

    # Loại bỏ khoảng trắng thừa do việc loại bỏ stopwords
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [11]:
clean_tex_ = clean_text(text, stopWord)

In [13]:
df['description'] = df['description'].apply(lambda x: clean_text(x, stopWord))

In [14]:
df['short_description'] = df['short_description'].apply(lambda x: clean_text(x, stopWord))

In [15]:
df['text_overview'] = df[['name', 'type', 'short_description', 'description']].agg(' '.join, axis=1)

In [16]:
print(df['name'][1])
print(df['type'][1])
print(df['short_description'][1])
print(df['description'][1])
print("================")
print(df['text_overview'][1])

Bảng trắng có chân gấp Flipchart F3 PolyTaiwan 60x100cm
simple
bảng trắng chân gấp flipchart thương hiệu bảng viết bavico chuyên sản xuất bảng viết việt nam sản phẩm bảng trắng chân gấp tinh tế tiện dụng hàng ưu chu
bảng trắng chân gấp flipchart thương hiệu bảng viết bavico chuyên sản xuất bảng viết việt nam sản phẩm bảng trắng chân gấp tinh tế tiện dụng hàng ưu chuộng bảng trắng mặt bảng trắng mặt bảng trắng viết bút lông poly taiwan viết bút lông lau sạch bóng vết mực kẻ ô ly lót bảng ván mdf ép giấy bóng vân gỗ chống ẩm chống cong mo bảng viết rung nhựa dầy ly chống chống cong mo khungbo bảng nhôm việt nhật chuyên dụng chống oxy hóa co góc bốn góc bảng co nhựa màu trắng sữa hài hòa chống trầy xước máng đựng bút nhôm chuyên dụng cm hai đầu co nhựa bảng gắn kẹp giấy inox bóng chống rỉ sétchân bảng trắng sắt sơn tĩnh điện chống sét chống trầy chân bảng xếp gọn phép chỉnh bảng chiều chân bảng gắn bánh xe taiwan di cố định dànggiá sản phẩm tiki bao thuế luật hiện hành cạnh tuỳ sản phẩm h

In [19]:
def calculate_tfidf(corpus):
    """
    Tính toán TF-IDF cho một tập các văn bản (corpus) và trả về các từ quan trọng.
    """
    # Khởi tạo vectorizer với giá trị TF-IDF
    vectorizer = TfidfVectorizer()

    # Tính toán TF-IDF cho toàn bộ corpus
    X = vectorizer.fit_transform(corpus)

    # Lấy ra các từ tương ứng với các cột trong ma trận TF-IDF
    feature_names = vectorizer.get_feature_names_out()

    # Trả về ma trận TF-IDF và danh sách các từ
    return X, feature_names

def filter_important_words(texts, X, feature_names, threshold=0.1):
    """
    Loại bỏ những từ không quan trọng dựa trên giá trị TF-IDF (threshold là ngưỡng để loại bỏ từ).
    Trả về văn bản hoàn chỉnh sau khi lọc từ không quan trọng.
    """
    important_texts = []
    
    for doc_index, text in enumerate(texts):
        # Tách từ trong văn bản
        words = text.split()

        # Lấy ra các giá trị TF-IDF tương ứng của văn bản hiện tại
        doc_tfidf = X[doc_index].toarray().flatten()

        # Giữ lại những từ có giá trị TF-IDF cao hơn threshold
        important_words = [
            word for word in words if word in feature_names and doc_tfidf[feature_names.tolist().index(word)] > threshold
        ]

        # Ghép lại các từ thành văn bản hoàn chỉnh
        # important_texts.append(" ".join(important_words))

    return important_words

In [27]:
corpus = df['description'].tolist()
print("=======\n")
X, feature_names = calculate_tfidf(corpus)
print("--------\n")
# Lọc các từ quan trọng
important_texts = filter_important_words(corpus, X, feature_names, threshold=0.1)


--------



In [30]:
def remove_words(text, words_to_remove):
    """
    Loại bỏ các từ trong danh sách words_to_remove khỏi đoạn văn bản text.
    """
    # Tách văn bản thành danh sách các từ
    words = text.split()
    
    # Loại bỏ các từ có trong danh sách words_to_remove
    filtered_words = [word for word in words if word.lower() not in words_to_remove]
    
    # Ghép lại các từ thành một chuỗi văn bản
    filtered_text = " ".join(filtered_words)
    
    return filtered_text

In [31]:
df['overview'] = df['description'].apply(lambda x: remove_words(x, important_texts))

In [32]:
df[['description', 'overview']]

,description,overview
0,học sinh thân mến biên soạn tập tiếng đáp án s...,học sinh thân mến biên soạn tập tiếng đáp án s...
1,bảng trắng chân gấp flipchart thương hiệu bảng...,bảng trắng chân gấp flipchart thương hiệu bảng...
2,biến kế toán khô khan trò trẻ áp dụng bao cầm ...,biến kế toán khô khan trò trẻ áp dụng bao cầm ...
3,thông tác phẩm tác phẩm combo tư ngược tư tác ...,thông tác phẩm tác phẩm combo tư ngược tư tác ...
4,thông tác phẩm tác phẩm combo tư ngược tư tác ...,thông tác phẩm tác phẩm combo tư ngược tư tác ...
...,...,...
779,tác giả tác giả xuất nxb đại học sư phạm trang...,tác giả tác giả xuất nxb đại học sư phạm trang...
780,thông tác giả song hong binh tống hồng binh si...,thông tác giả song hong binh tống hồng binh si...
781,phân tích chứng khoán sách tài ảnh hưởng giới ...,phân tích chứng khoán sách tài ảnh hưởng giới ...
782,hai mèo cửa sổ nhân vật gấu nhân vật hai tí ho...,hai mèo cửa sổ nhân vật gấu nhân vật hai tí ho...


In [33]:
from sentence_transformers import SentenceTransformer

# Load pre-trained Sentence Transformer model
embedding_model = SentenceTransformer("thenlper/gte-large")

def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempting to get embedding")
        return []
    embeddings = embedding_model.encode([text])
    return embeddings.tolist()

d:\WorkSpace\RAG\MyVenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [36]:
df['embedding'] = df['description'].apply(lambda x: get_embedding(x))

In [37]:
df.columns

Index(['_id', 'id', 'name', 'type', 'current_seller', 'images', 'short_url',
       'thumbnail_url', 'short_description', 'description', 'rating_average',
       'quantity_sold', 'embedding', 'text_overview', 'overview'],
      dtype='object')

In [39]:
df_dict = df[['id', 'name', 'type', 'current_seller', 'images', 'short_url',
       'thumbnail_url', 'short_description', 'description', 'rating_average',
       'quantity_sold', 'embedding', ]].to_dict('records')

In [41]:
db = MongoDBConnection(MONGODB_URL, DATABASE_NAME).get_database()
collection = db['product_new']
collection.insert_many(df_dict)

InsertManyResult([ObjectId('67069f4bb9def1690c530b9e'), ObjectId('67069f4bb9def1690c530b9f'), ObjectId('67069f4bb9def1690c530ba0'), ObjectId('67069f4bb9def1690c530ba1'), ObjectId('67069f4bb9def1690c530ba2'), ObjectId('67069f4bb9def1690c530ba3'), ObjectId('67069f4bb9def1690c530ba4'), ObjectId('67069f4bb9def1690c530ba5'), ObjectId('67069f4bb9def1690c530ba6'), ObjectId('67069f4bb9def1690c530ba7'), ObjectId('67069f4bb9def1690c530ba8'), ObjectId('67069f4bb9def1690c530ba9'), ObjectId('67069f4bb9def1690c530baa'), ObjectId('67069f4bb9def1690c530bab'), ObjectId('67069f4bb9def1690c530bac'), ObjectId('67069f4bb9def1690c530bad'), ObjectId('67069f4bb9def1690c530bae'), ObjectId('67069f4bb9def1690c530baf'), ObjectId('67069f4bb9def1690c530bb0'), ObjectId('67069f4bb9def1690c530bb1'), ObjectId('67069f4bb9def1690c530bb2'), ObjectId('67069f4bb9def1690c530bb3'), ObjectId('67069f4bb9def1690c530bb4'), ObjectId('67069f4bb9def1690c530bb5'), ObjectId('67069f4bb9def1690c530bb6'), ObjectId('67069f4bb9def1690c530b

In [66]:
def pipeline_processing(text, stopword, threshold=0.1):
    
    # lượt bỏ số và dấu 
    text = remove_numbers_and_special_chars(text)
    
    # xử lý văn bản
    clean_texts = clean_text(text, stopword)
    
    # tính toán TF-IDF
    cleaned_corpus = [clean_texts for doc in text]
    X, feature_names = calculate_tfidf(cleaned_corpus)
    
    # lọc từ quan trọng
    important_texts = filter_important_words(cleaned_corpus, X, feature_names, threshold=threshold)
    
    return important_texts[-1]

In [67]:
textComplete = pipeline_processing(text, stopword)

In [69]:
len(textComplete)

423